In [1]:
# 토큰 임베딩 - 토큰화 -> 토큰 아이디 부여 -> 토큰 임베딩층 토큰 임베딩
# 1. 토큰화 (Tokenization)
# 2. 토큰 임베딩으로 변환 - 토큰 아이디에서 벡터로 변환

# 위치 인코딩(position encoding) - 토큰화 -> 위치 아이디 -> 위치 인코딩층 위치 인코딩
# 1. 위치 인코딩 - 절대적 위치 인코딩

# 최종 입력 임베딩 - 토큰 임베딩 + 위치 인코딩

In [2]:
import torch
from torch import nn

In [3]:
# 2.1 토큰화 코드

# 띄어쓰기 단어로 분리
input_text = '나는 최근 파리 여행을 다녀왔다'
input_text_list = input_text.split()
print(input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = { word:idx for idx, word in enumerate(input_text_list) }
print('str2idx : ', str2idx)
idx2str = { idx:word for idx, word in enumerate(input_text_list) }
print('idx2str : ', idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [ str2idx[word] for word in input_text_list ]
print('input_ids : ', input_ids)

['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx :  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str :  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids :  [0, 1, 2, 3, 4]


In [4]:
# 토큰 아이디에서 벡터로 변환, 토큰 아이디(input_ids)를 16차원의 임의의 숫자 집합으로 변환
# - 임베딩 층이 단어의 의미를 담기 위해서는 딥러닝 모델이 학습 데이터로 훈련되어야 한다
# - 전체 모델이 학습되는 과정에서 임베딩 층도 학습된다.
import torch
from torch import nn

# nn.Embedding 클래스에 사전 크기가 len(str2idx) = 5
# embedding_dim = 16 차원
embedding_dim = 16
embed_layer = nn.Embedding(num_embeddings=len(str2idx), embedding_dim=embedding_dim)

input_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)
input_embeddings = input_embeddings.unsqueeze(0) # (배치차원,토큰,차원) (1, 5, 16), unsqueeze()함수를 사용하여 가장 왼쪽에 차원 추가
input_embeddings.shape

print(input_embeddings.shape)
print(input_embeddings)

torch.Size([1, 5, 16])
tensor([[[ 0.7304, -0.7336, -0.7616, -0.6751,  0.3791, -0.5551,  0.8373,
          -0.0984,  1.5027, -0.5485,  0.3170, -1.0218, -0.3052, -1.1560,
           0.1390,  0.0481],
         [ 1.4655,  1.0772,  0.7860, -1.0260,  0.6064, -0.2502,  0.0569,
          -0.7699,  0.8269, -0.0676,  0.1673, -1.2366, -0.7634,  0.5317,
          -0.5567, -0.2390],
         [-0.5011,  0.9714, -0.0293, -1.3315, -0.1914,  0.2001, -1.3953,
           0.1663,  0.3728,  0.9024,  3.0857, -1.6799, -0.4973,  0.9236,
          -0.6021,  0.1956],
         [-2.2506, -0.1843,  0.2845,  0.9180,  0.0203,  0.7818, -0.8363,
           0.5363, -0.7459, -0.6467, -0.3287,  1.0426, -0.1224, -1.3771,
          -1.3913,  0.8692],
         [-0.4913, -0.6655, -0.8549, -1.4564,  0.4976, -1.0587,  1.0498,
          -0.0605, -1.4870, -0.0768,  0.1009, -0.9007, -0.4111, -0.2142,
          -1.7900, -0.3155]]], grad_fn=<UnsqueezeBackward0>)


In [5]:
# 절대적 위치 인코딩
embedding_dim = 16
max_position = 12
embed_layer = nn.Embedding(len(str2idx), embedding_dim) # embedding layer
position_embed_layer = nn.Embedding(max_position, embedding_dim) # position layer

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids) # position encoding

token_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)
token_embeddings = token_embeddings.unsqueeze(0) # (1, 5, 16)

input_embeddings = token_embeddings + position_encodings
print(input_embeddings.shape)
print(input_embeddings)

torch.Size([1, 5, 16])
tensor([[[ 1.3184e-01,  7.7106e-01,  6.8728e-01, -2.4033e+00, -3.3944e-01,
          -2.9264e+00,  1.1917e+00,  9.8681e-01, -1.1124e+00, -2.9223e+00,
           1.2799e+00,  1.2936e+00, -1.5974e+00,  1.2628e+00,  8.3506e-01,
           1.5595e+00],
         [-1.4986e-01, -3.4427e+00,  5.5559e-01,  7.6686e-01,  1.4572e+00,
           6.5269e-01, -1.5479e+00, -5.1311e-01,  1.9119e+00, -1.3219e+00,
          -3.7489e-02, -1.3502e+00,  1.4470e+00, -6.2965e-01,  9.4782e-01,
          -1.6513e+00],
         [-3.1699e-02,  1.2296e-01, -2.1601e+00, -4.3810e-01,  1.6547e+00,
           3.2827e-03, -9.3602e-01,  1.3363e+00,  1.0686e+00,  2.0750e-01,
          -9.4739e-01,  2.6992e+00,  2.5093e-01, -7.0165e-01,  7.4804e-02,
           5.8139e-01],
         [ 5.5373e-01,  2.0975e+00,  1.4520e+00, -1.0624e-01,  7.0491e-01,
           3.1040e+00, -1.3199e+00, -1.5696e+00, -1.1113e+00, -1.6190e+00,
          -1.1090e+00,  6.0332e-01,  7.1202e-01, -1.9720e+00, -7.2001e-02,
     

In [ ]:
# 토큰 임베딩 - 토큰화 -> 토큰 아이디 부여 -> 토큰 임베딩층 토큰 임베딩
# 1. 토큰화 (Tokenization)
# 2. 토큰 임베딩으로 변환 - 토큰 아이디에서 벡터로 변환

# 위치 인코딩(position encoding) - 토큰화 -> 위치 아이디 -> 위치 인코딩층 위치 인코딩
# 1. 위치 인코딩 - 절대적 위치 인코딩

# 최종 입력 임베딩 - 토큰 임베딩 + 위치 인코딩

# 인코더(encoder)
# 1. 셀프 어텐션(Self Attention)
# 2. 멀티 헤더 어텐션(Multi-Head Attention)
# 3. 층 정규화 & 피드 포워드 층 (Normalization & Feed Forward Layer)

In [7]:
# Attention(어텐션) - 어텐션은 사람이 단어 사이의 관계를 고민하는 과정을 딥러닝 모델이 수행 할 수 있도록 모방한 연산이다
# 어텐션의 핵심 개념인 쿼리(query), 키(key), 값(value) - WQ, WK, WV 가중치 도입하고 학습 단계에서 업데이트 된다.
# 1) 쿼리(WQ)와 키(WK)의 관계를 계산한 관련도 값
# 2) 토큰 임베딩을 값 가중치(WV)로 변환한 값
# 어텐션은 1) + 2) 가중합을 하면 상관관계를 찾을수 있다

# 쿼리, 키, 값 벡터를 만드는 nn.Linear층
head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

# 변환 수행
querys = weight_q(input_embeddings) # (1, 5, 16
keys = weight_k(input_embeddings) # (1, 5, 16
values = weight_v(input_embeddings) # (1, 5, 16)

print(querys.shape)
print(keys.shape)
print(values.shape)

torch.Size([1, 5, 16])
torch.Size([1, 5, 16])
torch.Size([1, 5, 16])


In [8]:
print('keys 원본 모양 : ', keys.shape)
print('keys transpose 모양 : ', keys.transpose(-2, -1).shape)

keys 원본 모양 :  torch.Size([1, 5, 16])
keys transpose 모양 :  torch.Size([1, 16, 5])


In [ ]:
# 스케일 점곱 방식의 어텐션(Code of Scaled Dot-Product Attention)
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
    dim_k = querys.size(-1) # 16
    # 쿼리와 키를 곱한다
    # 이때 분산이 커지는 것을 방지하기 위해 임베딩 차원 수(dim_k)의 제곱근으로 나눈다
    # transpose(-2, -1) (1, 5, 16) -> (1, 16, 5)
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    
    # 쿼리와 키를 곱해 계산한 스코어(scores)를 합이 1이 되도록 소트트맥스(softmax)를 취해 가중치(weight)로 바꾼다
    weights = F.softmax(scores, dim=-1)
    
    # 가중치와 값을 곱해 입력과 동일한 형태의 출력을 반환(어텐션 연산 결과 값 반환)
    return weights @ values

In [10]:
# 어텐션 연산의 입력과 출력
print('원본 입력 형태 : ', input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print('어텐션 적용 후 형태 : ', after_attention_embeddings.shape)

원본 입력 형태 :  torch.Size([1, 5, 16])
어텐션 적용 후 형태 :  torch.Size([1, 5, 16])


In [ ]:
# 어텐션 연산을 수행하는 Attentionhead 클래스 생성
# 쿼리, 키, 값 벡터를 생성하기 위해 사용하는 선형층(weight_q, weight_k, weight_v)을 __init__ 메서드에서 생성하고
# forward 메서드에서는 앞서 생성한 선형층을 통해 쿼리, 키, 값 벡터를 생성하고 compute_attention 함수를 사용해 어텐션 연산을 수행한다
# 클래스를 사용할 때는 AttentionHead에 인자로 토큰 임베딩 차원(token_embed_dim)과 출력 차원(head_dim)을 전달하는데 입력과 출력이 같도록 둘다 embedding_dim을 전달했음
class AttentionHead(nn.Module):
    def __init__(self, token_embed_dim, head_dim, is_causal=False): # head_dim = Q, K, V의 차원을 나타냄
        super().__init__()
        self.is_causal = is_causal
        self.weight_q = nn.Linear(token_embed_dim, head_dim) # 쿼리 벡터 생성을 위한 선형 층
        self.weight_k = nn.Linear(token_embed_dim, head_dim) # 키 벡터 생성을 위한 선형 층
        self.weight_v = nn.Linear(token_embed_dim, head_dim) # 값 벡터 생성을 위한 선형 층
    
    def forward(self, querys, keys, values):
        outputs = compute_attention(
            self.weight_q(querys), # 쿼리 벡터
            self.weight_k(keys), # 키 벡터
            self.weight_v(values), # 값 벡터
            is_causal=self.is_causal
        )
        return outputs

attention_Head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_Head(input_embeddings, input_embeddings, input_embeddings)

print('attention_Head : ', attention_Head)
print('after_attention_embeddings : ', after_attention_embeddings)

attention_Head :  AttentionHead(
  (weight_q): Linear(in_features=16, out_features=16, bias=True)
  (weight_k): Linear(in_features=16, out_features=16, bias=True)
  (weight_v): Linear(in_features=16, out_features=16, bias=True)
)
after_attention_embeddings :  tensor([[[ 0.2191, -0.7716,  0.3835,  0.7176,  0.4080,  0.2059, -0.6205,
          -0.0189,  0.3360, -0.0228, -0.4808, -0.3776, -0.2003, -0.5566,
          -0.2033,  0.3038],
         [ 0.2147, -0.9281,  0.2782,  1.0232,  0.4651,  0.2570, -0.7754,
           0.1509,  0.4042,  0.0306, -0.5117, -0.4547, -0.0852, -0.2487,
          -0.3533,  0.3959],
         [ 0.0745, -0.6644,  0.7232,  0.7162,  0.2123,  0.2548, -0.4539,
          -0.0397,  0.2827,  0.0121, -0.1996, -0.2885,  0.0036, -0.5241,
          -0.1044,  0.5400],
         [ 0.2416, -1.2745,  0.7562,  1.3067,  0.2264,  0.6359, -1.1940,
          -0.0232,  0.9417,  0.5619, -0.4679, -0.5437, -0.1796, -0.7354,
          -0.1510,  0.8262],
         [-0.0777, -0.3613,  0.9606,  0.

In [ ]:
# Multi-Head Attention(멀티 헤드 어텐션) 구현
# 하나의 어텐션만 수행하는게 아니라 여러 어텐션 연산을 동시에 적용하면 성능을 더 높일 수 있다는 사실을 발견했다 이를 멀티 헤드 어텐션이라 한다
# 토큰 사이의 관계를 한 가지 측면에서 이해하는 것보다 여러 측면을 동시에 고려할때 언어나 문장에 대한 이해도가 높아질 것이다.

class MultiheadAttention(nn.Module):
    def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
        super().__init__()
        self.n_head = n_head
        self.is_causal = is_causal
        self.weight_q = nn.Linear(token_embed_dim, d_model)
        self.weight_k = nn.Linear(token_embed_dim, d_model)
        self.weight_v = nn.Linear(token_embed_dim, d_model)
        self.concat_linear = nn.Linear(d_model, d_model)
    
    def forward(self, querys, keys, values):
        B, T, C = querys.size() # 배치, 토큰, 차원
        # 헤드의 수(n_head) 만큼 쿼리,키,값을 만들고 연산을 수행 - 쿼리, 키, 값을 n_head개로 나눈 몫, 1->2 순서 변경
        querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        # 각각의 어텐션을 계산
        attention = compute_attention(querys, keys, values, self.is_causal)

        # 입력과 같은 형태로 변환해주면서 contiguous() 함수를 사용하여 메모리 할달 일정하게 해 줌
        output = attention.transpose(1, 2).contiguous().view(B, T, C)
        # 마지막으로 선형층을 통화시키고 최종 결과를 반환한다
        output = self.concat_linear(output)
        return output

n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)

print(after_attention_embeddings.shape)

torch.Size([1, 5, 16])


In [ ]:
# 토큰 임베딩 - 토큰화 -> 토큰 아이디 부여 -> 토큰 임베딩층 토큰 임베딩
# 1. 토큰화 (Tokenization)
# 2. 토큰 임베딩으로 변환 - 토큰 아이디에서 벡터로 변환

# 위치 인코딩(position encoding) - 토큰화 -> 위치 아이디 -> 위치 인코딩층 위치 인코딩
# 1. 위치 인코딩 - 절대적 위치 인코딩

# 최종 입력 임베딩 - 토큰 임베딩 + 위치 인코딩

# 인코더(encoder)
# 1. 셀프 어텐션(Self Attention)
# 2. 멀티 헤더 어텐션(Multi-Head Attention)
# 3. 층 정규화 & 피드 포워드 층 (Normalization & Feed Forward Layer)

In [14]:
# 층 정규화 & 피드 포워드 층(Normalization & Feed Forward Layer)
# 정규화(Normalization)
# - 딥러닝 모델에서 입력이 일정한 분포를 갖도록 만들어 학습이 안정적이고 빨라질 수 있도록 함
# - norm_x = (x - 평균(u) / 표준편차 => 평균이 0이고 표준편차 1로 바꿈
# - 배치 정규화(batch normalization) : 배치 입력 데이터 사이에 정규화 수행(예 이미지 처리)
# - 층 정규화(layer normalization) : 특징 차원에서 정규화를 수행(예 자연어 처리, 문장의 토큰 개수가 각각 다르므로 층 정규화가 적합함)

# 층 정규화 코드
norm = nn.LayerNorm(embedding_dim) # 16
norm_x = norm(input_embeddings)
print(norm_x.shape)

norm_x.mean(dim=-1).data, norm_x.std(dim=-1) # 실제로 평균과 표준편차 확인하기

torch.Size([1, 5, 16])


(tensor([[-1.4901e-08, -1.4901e-08, -7.4506e-09,  1.4901e-08,  4.4703e-08]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]], grad_fn=<StdBackward0>))

In [15]:
# 피드 포워드 층(Feed Forward Layer)
# - 데이터의 특징을 학습하는 완전 연결 층(fully connected layer) 말한다
# - 멀티 헤드 어텐션이 단어 사이의 관계를 파악하는 역할이라면 피드 포워드 층은 입력 텍스트 전체를 이해하는 역할을 담당한다
# - 선형 층, 드롭아웃 층, 층 정규화, 활성 함수로 구성된다
# - 임베딩의 차원을 동일하게 유지해야 쉽게 층을 쌓아 확장이 가능하기 때문에 입력과 출력의 형태가 동일하도록 맞춘다
# - 일반적으로 d_model 차원에서 d_model 보다 2~3배 큰 dim_feedforward 차원으로 확장했다가 다시 d_model로 변환한다

# 피드 포워드 층 코드
class PreLayerNormFeedForward(nn.Module):
    def __init__(self, d_model, dim_feedforward, dropout):
        super().__init__()
        self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 1층
        self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 2층
        self.dropout1 = nn.Dropout(dropout) # 드롭아웃 1층
        self.dropout2 = nn.Dropout(dropout) # 드롭아웃 2층
        self.activation = nn.GELU() # 활성 함수
        self.norm = nn.LayerNorm(d_model) # 층 정규화
    
    def forward(self, src):
        x = self.norm(src) # 층 정규화
        # feed forward & residual connection, 기존 입력값을 더해준다(기존 값이 희미해지거나 기울기 소실 방지 차원)
        x = x + self.linear2(self.dropout1(self.activation(self.linear1(x)))) # 선형 1층 -> 활성 함수 -> 드롭아웃 1층 -> 선형 2층
        x = self.dropout2(x) # 드롭아웃 2층
        return x

In [16]:
# 인코더(encoder)
# - 인코더는 멀티 헤드 어텐션, 층 정규화, 피드 포워드 층이 반복되는 형태다
# - 안정적인 학습이 가능하도록 도와주는 잔차 연결(residual connection)이다, 잔차 연결은 입력을 다시 더해주는 형태로 구현한다

# 인코더 층 코드
class TransformerEncoderLayer(nn.Module):
    # 멀티 헤드 어텐션, 층 정규화, 드롭아웃, 피드 포워드 층을 불러온다
    def __init__(self, d_model, nhead, dim_feedforward, dropout):
        super().__init__()
        self.attn = MultiheadAttention(d_model, d_model, nhead) # 멀티 헤드 어텐션 클래스
        self.norm1 = nn.LayerNorm(d_model) # 층 정규화
        self.dropout1 = nn.Dropout(dropout) # 드롭 아웃
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) # 피드 포워드
    
    def forward(self, src):
        norm_x = self.norm1(src) # 층 정규화
        attn_output = self.attn(norm_x, norm_x, norm_x) # 멀티 헤드 어텐션 클래스를 인스턴스화한 self.attn을 통해 멀티 헤드 어텐션 연산 수행
        x = src + self.dropout1(attn_output) # 잔차 연결을 위해 어텐션 결과에 드롭아웃을 취한 self.dropout1(attn_output)과 입력(src)을 더해준다.

        # 피드 포워드
        x = self.feed_forward(x) # self.feed_forward(x)를 통해 피드 포워드 연산을 수행
        return x


In [19]:
# 인코더 구현
import copy

# get_clones() 함수는 입력한 모듈을 깊은 복사를 통해 N번 반복해 모듈 리스트를 담는다.
def get_clones(module, N):
    return nn.ModuleList( [ copy.deepcopy(module) for i in range(N) ] )

# TransformerEncoder 클래스에서는 인자로 전달 받은 encoder_layer를 get_clones() 함수를 통해 num_layers번 반복해
# nn.ModuleList에 넣고 forward 메서드에서 for문을 통해 순회하면서 인코더 층 연산을 반복 수행한다
class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers):
        super().__init__()
        self.layers = get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm
    
    def forward(self, src):
        output = src
        for mod in self.layers:
            output = mod(output)
        return output

In [ ]:
# 입력값 -> 토큰 임베딩 -> 위치 인코딩 -> 최종 임베딩값
# 토큰 임베딩 - 토큰화 -> 토큰 아이디 부여 -> 토큰 임베딩층 토큰 임베딩
# - 토큰화 (Tokenization)
# - 토큰 임베딩으로 변환 - 토큰 아이디에서 벡터로 변환
# 위치 인코딩(position encoding) - 토큰화 -> 위치 아이디 -> 위치 인코딩층 위치 인코딩
# - 위치 인코딩 - 절대적 위치 인코딩
# 최종 입력 임베딩 - 토큰 임베딩 + 위치 인코딩

# 입력값 -> 토큰 임베딩 -> 위치 인코딩 -> 최종 임베딩값 => 인코더
# 인코더(encoder)
# 1. 셀프 어텐션(Self Attention)
# 2. 멀티 헤더 어텐션(Multi-Head Attention)
# 3. 층 정규화 & 피드 포워드 층 (Normalization & Feed Forward Layer)

# 입력값 -> 토큰 임베딩 -> 위치 인코딩 -> 최종 임베딩값 => 디코더
# 입력값 -> 토큰 임베딩 -> 위치 인코딩 -> 최종 임베딩값 => 인코더 => 디코더
# 디코더(decoder) - 2곳으로 부터 입력값을 받아서 수행
# 1. 마스크 멀티 헤드 어텐션(Masked Muli-Head Attention)

In [ ]:
# 디코더에서 어텐션 연산(마스트 어텐션) 코드
# 디코더는 생성을 담당하는 부분, 사람이 글을 쓸때 앞 단어부터 순차적으로 작성하는 것처럼 트랜스포머 모델도 앞에서 생성한 토큰을 기반으로 다음 토큰을 생성한다
# 실제 텍스트를 생성할때 디코더는 이전까지 생성한 텍스트만 확인 할 수 있다, 그런데 학습할때는 인코더나 디코더 모두 완성된 텍스트를 입력으로 받는다.
# 언텐션을 그대로 활용할 경우 미래 시점에 작성해야하는 텍스트를 미리 확인하게 되면 문제가 생긴다, 이를 막기 위해 특점 시점에는 그 이전의 생성된 토큰까지만
# 확인 할 수 있도록 마스크를 추가한다
def compute_attention(querys, keys, values, is_causal=False):
    dim_k = querys.size(-1) # 16
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k) # (1, 5, 5)
    
    # is_causal = True 일때 masking이  가능하게 설정 
    if is_causal:
        query_length = querys.size(-2)
        key_length = keys.size(-2)
        temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0) # torch.triu(upper 상삼각형), torch.tril(lower 하삼각형)
        scores = scores.masked_fill(temp_mask == False, float('-inf')) # temp_mask == False -> -inf(음의 무한대) 변경
    weights = F.softmax(scores, dim=-1) # (1, 5, 5)
    return weights @ values # (1, 5, 5)

In [45]:
# 크로스 어텐션(Cross Attention)
# - 크로스 어텐션은 인코더의 결과값과 디코더의 마스크 멀티헤더 어텐션 결과값 2가지를 입력으로 받아서 수행
# - Encoder output -> Decoder Cross Attention <- Decoder Masked Multi-Head Attention output
# 예시) 영어에서 한국어로 번역한다고 했을때 인코더가 영어 문장을 입력으로 받아 처리한 결과를 번역한 한국어를 생성하는 디코더가 받아 활용한다.
# 이때 쿼리는 디코더의 잠재 상태를 사용하고 키와 값은 인코더의 결과를 사용한다

# 크로스 어텐션 포함된 디코드 층 코드
class TransformerDecoder(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.self_attn = MultiheadAttention(d_model, d_model, nhead) # 인코더
        self.multihead_attn = MultiheadAttention(d_model, d_model, nhead) # 디코더
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) # 피드 포워드

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
    
    # tgt(output embedding), encoding output 모두 input으로 받음
    def forward(self, tgt, encoder_output, is_causal=True):
        # Self Attention 연산
        x = self.norm1(tgt) # output embedding vector 정규화
        x = x + self.dropout1(self.self_attn(x, x, x, is_causal = is_causal)) # is_causal = True 하여 masked attention 수행

        # Cross Attention 연산, 디코더 쿼리, 인코더 키, 인코더 값
        x = self.norm2(x) # decoder attention 정규화
        x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output)) # cross attention 및 dropout 수행

        # Feed Forward 연산
        x = self.feed_forward(x) # FeedForwardLayer 통과

        return x

In [48]:
# 디코더 구현 코드
import copy

# 디코더 층을 N번 반복 nn.ModuleList를 활용해 순회하도록 수행
def get_clones(module, N):
    return nn.ModuleList( [ copy.deepcopy(module) for i in range(N) ] )

class TransformerDecoder(nn.Module):
    def __init__(self, decoder_layer, num_layers):
        super().__init__()
        self.layers = get_clones(decoder_layer, num_layers)
        self.num_layers = num_layers
    
    def forward(self, tgt, src):
        output = tgt
        for mod in self.layers:
            output = mod(tgt, src)
        return output